In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. 기존 리포지토리 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project')

In [3]:
# 2. 경로 확인
!pwd

/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project


In [4]:
import os
import shutil
import json
import random
from pathlib import Path
from typing import Dict, List, Tuple
import glob

In [5]:
def find_annotation_file(image_filename: str, annotations_root: str) -> str:
    """
    이미지 파일명에 해당하는 annotation json 파일을 찾습니다.
    """
    # 이미지 파일명에서 확장자를 제거하고 .json을 붙임
    json_filename = image_filename.replace('.png', '.json').replace('.jpg', '.json')

    # annotations_root에서 해당 json 파일을 재귀적으로 찾기
    pattern = os.path.join(annotations_root, '**', json_filename)
    matches = glob.glob(pattern, recursive=True)

    if matches:
        return matches[0]  # 첫 번째 매치를 반환
    else:
        return None

In [6]:
def create_dataset_subset(
    source_dir: str,
    target_dir: str,
    subset_name: str,
    num_samples: int,
    seed: int = 42
) -> Dict[str, int]:
    """
    원본 데이터셋에서 지정된 수만큼 샘플을 추출하여 새로운 데이터셋을 생성합니다.

    Args:
        source_dir: 원본 데이터셋 경로
        target_dir: 생성될 서브셋 데이터셋 경로
        subset_name: 서브셋 이름 (small, medium, large)
        num_samples: 추출할 샘플 수
        seed: 랜덤 시드

    Returns:
        생성된 파일 수 정보가 담긴 딕셔너리
    """
    random.seed(seed)

    # 경로 설정
    train_images_dir = os.path.join(source_dir, 'train_images')
    train_annotations_dir = os.path.join(source_dir, 'train_annotations')
    test_images_dir = os.path.join(source_dir, 'test_images')

    # 타겟 디렉토리 생성
    target_subset_dir = os.path.join(target_dir, f'dataset_{subset_name}')
    target_train_images = os.path.join(target_subset_dir, 'train_images')
    target_train_annotations = os.path.join(target_subset_dir, 'train_annotations')
    target_test_images = os.path.join(target_subset_dir, 'test_images')

    # 디렉토리 생성
    os.makedirs(target_train_images, exist_ok=True)
    os.makedirs(target_train_annotations, exist_ok=True)
    os.makedirs(target_test_images, exist_ok=True)

    # train 이미지 파일 목록 가져오기
    train_image_files = []
    for ext in ['*.png', '*.jpg', '*.jpeg']:
        train_image_files.extend(glob.glob(os.path.join(train_images_dir, ext)))

    print(f"총 {len(train_image_files)}개의 train 이미지를 찾았습니다.")

    # 랜덤하게 샘플 선택
    if num_samples > len(train_image_files):
        print(f"경고: 요청한 샘플 수({num_samples})가 전체 데이터 수({len(train_image_files)})보다 큽니다.")
        num_samples = len(train_image_files)

    selected_images = random.sample(train_image_files, num_samples)

    # 통계
    stats = {
        'copied_images': 0,
        'copied_annotations': 0,
        'missing_annotations': 0,
        'copied_test_images': 0
    }

    print(f"\n{subset_name} 데이터셋 생성 중... ({num_samples}개 샘플)")

    # 선택된 이미지들과 해당 어노테이션 복사
    for img_path in selected_images:
        img_filename = os.path.basename(img_path)

        # 이미지 복사
        target_img_path = os.path.join(target_train_images, img_filename)
        shutil.copy2(img_path, target_img_path)
        stats['copied_images'] += 1

        # 해당하는 어노테이션 파일 찾기
        annotation_path = find_annotation_file(img_filename, train_annotations_dir)

        if annotation_path:
            # 원본 어노테이션 폴더 구조 유지
            # 예: train_annotations/K-003544-010221-016551-027926_json/K-003544/file.json
            rel_path = os.path.relpath(annotation_path, train_annotations_dir)
            target_annotation_path = os.path.join(target_train_annotations, rel_path)

            # 타겟 디렉토리 생성
            os.makedirs(os.path.dirname(target_annotation_path), exist_ok=True)

            # 어노테이션 파일 복사
            shutil.copy2(annotation_path, target_annotation_path)
            stats['copied_annotations'] += 1
        else:
            print(f"경고: {img_filename}에 해당하는 어노테이션 파일을 찾을 수 없습니다.")
            stats['missing_annotations'] += 1

    # 테스트 이미지들도 모두 복사 (전체 테스트 세트 유지)
    if os.path.exists(test_images_dir):
        for test_img in glob.glob(os.path.join(test_images_dir, '*')):
            if os.path.isfile(test_img):
                shutil.copy2(test_img, os.path.join(target_test_images, os.path.basename(test_img)))
                stats['copied_test_images'] += 1

    return stats

In [7]:
def create_all_subsets(source_dir: str, target_dir: str, configs: Dict[str, int]):
    """
    모든 서브셋을 생성합니다.

    Args:
        source_dir: 원본 데이터셋 경로
        target_dir: 생성될 서브셋 데이터셋 경로
        configs: {subset_name: num_samples} 형태의 딕셔너리
    """
    print("="*60)
    print("데이터셋 분할 시작")
    print("="*60)

    total_stats = {}

    for subset_name, num_samples in configs.items():
        print(f"\n[{subset_name.upper()}] 데이터셋 생성 중...")
        stats = create_dataset_subset(source_dir, target_dir, subset_name, num_samples)
        total_stats[subset_name] = stats

        print(f"✅ {subset_name} 데이터셋 완성!")
        print(f"   - 이미지: {stats['copied_images']}개")
        print(f"   - 어노테이션: {stats['copied_annotations']}개")
        print(f"   - 테스트 이미지: {stats['copied_test_images']}개")
        if stats['missing_annotations'] > 0:
            print(f"   - 누락된 어노테이션: {stats['missing_annotations']}개")

    print("\n" + "="*60)
    print("모든 데이터셋 분할 완료!")
    print("="*60)

    return total_stats

In [8]:
def create_percentage_based_subsets(source_dir: str, target_dir: str, percentages: List[Tuple[str, float]]):
    """
    비율 기반으로 데이터셋 서브셋들을 생성합니다.

    Args:
        source_dir: 원본 데이터셋 경로
        target_dir: 생성될 서브셋 데이터셋 경로
        percentages: [(subset_name, percentage), ...] 형태의 리스트
                    예: [("nano", 0.1), ("small", 0.2), ("medium", 0.5), ("large", 0.8)]
    """
    # 원본 train 이미지 수 확인
    train_images_dir = os.path.join(source_dir, 'train_images')
    train_image_files = []
    for ext in ['*.png', '*.jpg', '*.jpeg']:
        train_image_files.extend(glob.glob(os.path.join(train_images_dir, ext)))

    total_train_images = len(train_image_files)

    print(f"원본 Train 이미지 수: {total_train_images}개")

    # 비율에 따른 데이터셋 크기 계산
    dataset_configs = {}
    print(f"\n생성될 데이터셋 크기:")
    for name, percentage in percentages:
        size = int(total_train_images * percentage)
        dataset_configs[name] = size
        print(f"  - {name}: {size}개 ({percentage*100:.0f}%)")

    print("\n" + "="*60)
    print("비율 기반 데이터셋 분할 시작")
    print("="*60)

    # 모든 서브셋 생성
    return create_all_subsets(source_dir, target_dir, dataset_configs)

In [9]:
if __name__ == "__main__":
    # 설정
    SOURCE_DATA_DIR = "./data"  # 원본 데이터 경로 (Colab 기준)
    TARGET_DATA_DIR = "./data/datasets"  # 생성될 데이터셋들의 상위 경로

    # 타겟 디렉토리 생성
    os.makedirs(TARGET_DATA_DIR, exist_ok=True)

    # 방법 1: 비율 기반으로 생성 (권장)
    percentages = [
        ("nano", 0.1),    # 10%
        ("small", 0.2),   # 20%
        ("medium", 0.5),  # 50%
        ("large", 0.8)    # 80%
    ]

    results = create_percentage_based_subsets(SOURCE_DATA_DIR, TARGET_DATA_DIR, percentages)

    # 최종 결과 출력
    print("\n📊 최종 통계:")
    for subset_name, stats in results.items():
        print(f"{subset_name}: {stats['copied_images']}개 이미지, {stats['copied_annotations']}개 어노테이션")

원본 Train 이미지 수: 1489개

생성될 데이터셋 크기:
  - nano: 148개 (10%)
  - small: 297개 (20%)
  - medium: 744개 (50%)
  - large: 1191개 (80%)

비율 기반 데이터셋 분할 시작
데이터셋 분할 시작

[NANO] 데이터셋 생성 중...
총 1489개의 train 이미지를 찾았습니다.

nano 데이터셋 생성 중... (148개 샘플)


KeyboardInterrupt: 

## 데이터셋 생성 시간
1. NANO: 12분
2. SMALL: 14분
3. MEDIUM: 28분
4. LARGE: 34분